In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
raw = pd.read_csv("books_Niek.csv", na_values='None')

In [3]:
start_time = time.time()
reviews = pd.read_csv("Books_rating.csv", na_values='None')
end_time = time.time()
print("Import time:", end_time - start_time, "seconds")

Import time: 21.129158973693848 seconds


In [4]:
# Change the values of missing
missing_val = raw["authors"][212400]
reviews = reviews.replace(missing_val, None)
raw = raw.replace(missing_val, None)

## Clean the raw file

In [5]:
# Remove the outer square brackets. Remove all quote marks. (Note some fields have multiple sets of quotes, if there are multiple authors)
def clean_author(str):
    if str is None:
        return None
    else:
        return ((str[1:-1]).replace("'", ""))

raw["authors"] = raw["authors"].apply(clean_author)

In [6]:
del raw["description"]
del raw["image"]
del raw["previewLink"]
del raw["infoLink"]
del raw["ratingsCount"]

In [7]:
def my_count(str):
    if str == None:
        return None
    else:
        return str.count("'")
        
raw["categories_count"] = raw["categories"].apply(my_count)

In [8]:
raw[raw["categories_count"] > 2.0]

# No book has more than 1 category, so let's safely remove the quotes and brackets like we did for authors

,Title,authors,publisher,publishedDate,categories,categories_count


In [9]:
raw["categories"] = raw["categories"].apply(clean_author)
del raw["categories_count"]

In [10]:
raw

,Title,authors,publisher,publishedDate,categories
0,Its Only Art If Its Well Hung!,Julie Strain,None,1996,Comics & Graphic Novels
1,Dr. Seuss: American Icon,Philip Nel,A&C Black,2005-01-01,Biography & Autobiography
2,Wonderful Worship in Smaller Churches,David R. Ray,None,2000,Religion
3,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,2005-02,Fiction
4,"Nation Dance: Religion, Identity and Cultural ...",Edward Long,None,2003-03-01,None
...,...,...,...,...,...
212399,The Orphan Of Ellis Island (Time Travel Advent...,Elvira Woodruff,Scholastic Paperbacks,2000-06-01,Juvenile Fiction
212400,Red Boots for Christmas,None,None,1995,Juvenile Fiction
212401,Mamaw,Wild Wild Cabbage,None,2018-01-17,None
212402,The Autograph Man,Zadie Smith,Vintage,2003-08-12,Fiction


## Clean the reviews file

In [11]:
del reviews["review/text"]

In [12]:
reviews

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary
0,1882931173,Its Only Art If Its Well Hung!,None,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images
1,0826414346,Dr. Seuss: American Icon,None,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It
2,0826414346,Dr. Seuss: American Icon,None,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library
3,0826414346,Dr. Seuss: American Icon,None,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment
4,0826414346,Dr. Seuss: American Icon,None,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview
...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,None,None,None,14/19,4.0,937612800,Difficult
2999996,B000NSLVCU,The Idea of History,None,A1SMUB9ASL5L9Y,jafrank,1/1,4.0,1331683200,Quite good and ahead of its time occasionally
2999997,B000NSLVCU,The Idea of History,None,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",0/0,4.0,1180224000,Easier reads of those not well versed in histo...
2999998,B000NSLVCU,The Idea of History,None,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",1/11,5.0,1163030400,"Yes, it is cheaper than the University Bookstore"


## Add review data to the raw data

In [13]:
average_per_title = reviews.groupby("Title")["review/score"].mean()

In [14]:
type(reviews["review/score"].iloc[2])

numpy.float64

In [15]:
print(average_per_title)

Title
" Film technique, " and, " Film acting "                                                                                                   4.500000
" We'll Always Have Paris": The Definitive Guide to Great Lines from the Movies                                                            5.000000
"... And Poetry is Born ..." Russian Classical Poetry                                                                                      4.000000
"A Titanic hero" Thomas Andrews, shipbuilder                                                                                               4.875000
"A Truthful Impression of the Country": British and American Travel Writing in China, 1880-1949                                            4.000000
                                                                                                                                             ...   
with an everlasting love                                                                                  

In [16]:
my_str = average_per_title.iloc[-3]

print(any(i == my_str for i in reviews["Title"]))
print(any(i == my_str for i in reviews["profileName"]))

False
False


In [17]:
# Sample DataFrame
data = {'Category': ['A', 'A', 'B', 'B', 'A'],
        'junk': ["stras", None, "a", "b", None],
        'Value': [10, 20, 30, None, 50]}
df = pd.DataFrame(data)

average_per_cat = df.groupby("Category")["Value"].mean()
print(average_per_cat)
print(df)

Category
A    26.666667
B    30.000000
Name: Value, dtype: float64
  Category   junk  Value
0        A  stras   10.0
1        A   None   20.0
2        B      a   30.0
3        B      b    NaN
4        A   None   50.0
